## Genearete labels

In [1]:
import pandas as pd
import nutils
import common as cm
import numpy as np
import os
from tqdm import tqdm


def calculate_stats(df, current, future):
    log_returns = np.log(df["mid_price"].shift(-60) / df["mid_price"]) * 1e4
    df[f"mean_{current}-{future}"] = (
        np.log(
            (df["mid_price"].rolling(future - current).mean().shift(-future))
            / df["mid_price"]
        )
        * 1e4
    )
    # df[f"var_{current}-{future}"] = (
    #     log_returns.rolling(future - current).var().shift(-future)
    # )
    # df[f"vol_{current}-{future}"] = (
    #     log_returns.rolling(future - current).std().shift(-future)
    # )


def calculate_stats2(df, current, future):
    df[f"max_{current}-{future}"] = (
        np.log(
            (df["mid_price"].rolling(future - current).max().shift(-future))
            / df["mid_price"]
        )
        * 1e4
    )
    df[f"min_{current}-{future}"] = (
        np.log(
            df["mid_price"].rolling(future - current).min().shift(-future)
            / df["mid_price"]
        )
        * 1e4
    )
    df[f"gap_{current}-{future}"] = (
        (
            df["mid_price"].rolling(future - current).max().shift(-future)
            - df["mid_price"].rolling(future - current).min().shift(-future)
        )
        / df["mid_price"]
        * 1e4
    )


for code in tqdm(cm.SELECTED_CODES):
    datas = cm.get_snapshot(code)
    df = datas["tickData"]
    df = pd.DataFrame(df, columns=cm.COLS_SNAPSHOTS)

    # 保存原始列名
    original_columns = set(df.columns)
    df["mid_price"] = (df["AskPrice1"] + df["BidPrice1"]) / 2
    cur_futs = [(0, 60), (60, 120), (60, 300), (60, 600), (60, 6000), (120, 180)]
    for cur, futs in cur_futs:
        # calculate_stats(df, cur, futs)
        calculate_stats2(df, cur, futs)

    # 只保留新生成的列
    new_columns = set(df.columns) - original_columns
    df = df.fillna(0)

    # 指定保存路径
    save_path = f"/mnt/disk1/multiobj_dataset/{code}/"
    os.makedirs(save_path, exist_ok=True)
    for col in new_columns:
        feature_path = os.path.join(save_path, f"{col}.npy")
        np.save(feature_path, df[col].values.reshape(-1, 1))
print(new_columns)

100%|██████████| 100/100 [21:40<00:00, 13.00s/it]

{'gap_60-300', 'min_60-600', 'gap_60-120', 'min_120-180', 'min_60-6000', 'max_120-180', 'min_60-120', 'min_0-60', 'mid_price', 'gap_60-6000', 'max_60-6000', 'gap_120-180', 'gap_60-600', 'max_0-60', 'max_60-120', 'gap_0-60', 'max_60-300', 'min_60-300', 'max_60-600'}
